In [32]:
def GetBinaryPhoto(input_file : str) -> list[str]:

    # Čitanje slika kao binarnog koda
    with open(input_file, 'rb') as photo:
        read = photo.read()
    read = list(read)

    # Podela bajtova na bitove
    bits_data = []
    for byte in read:
        bits = format(byte, '08b')
        bits_data.append(bits[:2])
        bits_data.append(bits[2:4])
        bits_data.append(bits[4:6])
        bits_data.append(bits[6:])

    return bits_data

photo_bits = GetBinaryPhoto('fotka.jpg')

## Coding audio

In [65]:
import wave

def ModifyBits(byte : bytes, bits : str) -> bytes:
    bits = int(bits, 2)
    return (byte & 0b11111100) | bits

def ModifyWav(input_filename : str, output_filename : str, bits : list) -> None:

    # Čitanje audio fajla
    with wave.open(input_filename, 'rb') as infile:
        params = infile.getparams()
        frames = infile.readframes(params.nframes)
    
    # Provera da li slika može da stane u audio fajl
    if len(bits) > len(frames) - 24: raise MemoryError("Slika je previše velika da bi stala u audio fajl")
    
    # Sredjivanje tipa podataka
    frames = bytearray(frames)

    # Sakrivanje dužine slike
    num_bytes = len(bits) // 4
    photo_length = format(num_bytes, '024b')
    length_bits = []
    for i in range(0, len(photo_length), 2):
        length_bits.append(photo_length[i : i + 2])
    writen_bits = 0
    for i in range(1, 24, 2):
        frames[i] = ModifyBits(frames[i], length_bits[writen_bits])
        writen_bits += 1
    
    # Sakrivanje slike
    writen_bits = 0
    for i in range(25, len(frames), 2):
        if writen_bits < len(bits):
            new_bits = ModifyBits(frames[i], bits[writen_bits])
            writen_bits += 1
            frames[i] = new_bits
    
    # Pravljenje novog fajla
    with wave.open(output_filename, 'wb') as outfile:
        outfile.setparams(params)
        outfile.writeframes(frames)

input_filename = 'nikotin-96khz.wav'
output_filename = 'output.wav'
ModifyWav(input_filename, output_filename, photo_bits)


## Decoding audio

In [67]:
import wave

def GetBits(byte):
    ans = f'{byte & 0b00000011 : 03b}'[1:]
    return ans

def DemodulateSignal(input_filename, output_filename):
    with wave.open(input_filename, 'rb') as infile:
        params = infile.getparams()
        frames = infile.readframes(params.nframes)

    frames = bytearray(frames)

    photo_lenght = ''
    for i in range(1, 24, 2):
        photo_lenght += GetBits(frames[i])
    num_bytes = int(photo_lenght, 2)
    num_bits = num_bytes * 4

    s = ''
    for i in range(num_bits):
        s += GetBits(frames[i * 2 + 25])
    
    data = []
    for i in range(0, len(s), 8):
        data.append(bytes(s[i : i + 8], 'utf-8'))

    with open(output_filename, 'wb') as slika:
        for thing in data:
            slika.write(int(thing, 2).to_bytes())

    
DemodulateSignal('output.wav', 'slika.jpg')

In [44]:
s = 0b10
print(format(s, '024b'))

000000000000000000000010
